# Making Databases from Previous Versions' Data
- <b>Name:</b> Sofia Kobayashi
- <b>Date:</b> 01/02/2023
- <b>Notebook Stage:</b> 1.0 (initial data collection & cleaning), finished
- <b>Description:</b> Collecting all the data stored in all previous versions (v9.1 and below), and trying to collate all data, sort by work type & make into databases
    - ONLY url data added here, much text data will be added in future notebooks

### **<u>Table of Contents</u>**

1. **[Imports](#imports)**
1. **[Load in ALL urls](#load_urls)**
    - All urls up until 01/13/2023
1. **[Create 4 (sources) Database](#4_dtb)**
    - v7_sheets, v8_old_local_files, pre (urls collected before 1/13/23), cur (urls collected on 1/13/23)
    - cols: url, dtb_type, date_added, date_last_viewed, smk_source
1. **[Combine 4 (sources) DTBs -> 1 big DTB (of all URLs)](#4_to_1_dtb)**
1. **[Adding/filling metadata cols](#m1)**
    - filled date_added, added version num
1. **[Seperate AO3 works vs Others](#seperate_others)**
    - **Others CHECKPOINT 0** - all non-AO3 urls & AO3 external works
1. **[Label AO3 URLs & Separate into (work) DTBs](#seperate_ao3_dtbs)**
1. **[Clean up & customize the new 7 databases](#clean_7)**
    - 7 DTBs: collections, comments, search, series, tags, users, works
    - **Col, Comments, Search, Tags, Users CHECKPOINTS 0** - in 'data-checkpoints' folder
1. **[Fill seriesDTB](#fill_series)**
    - **Series CHECKPOINT 1** - all series urls (as of 1/13/23) w/ basic meta data, de-dupped
        - did not add text-data series
1. **[Clean & prepare ficDTB](#fill_fics)**
    - [**Fic CHECKPOINT 1**](#fill_fics_1) - *all* fic urls w/ basic metadata
        - Cols: dtb_type, smk_source, version, date_added, date_last_viewed, url_type, id, url, col_work
    - [**Fic CHECKPOINT 2**](#fill_fics_2) - fic urls fic-de-dupped (no fic info)
        - Added cols: url_psueds, cur_chapter, notes
    - [**Fic CHECKPOINT 3**](#fill_fics_3) - reorganized cols, added more, prepared for adding fic info (no fic info)
        - Added cols: dtb_type, smk_source, version	date_added	date_last_viewed	url_type	id	url	col_work	url_psueds	cur_chapter	notes	title	authors	fandoms	fic_obj	date_obj_updated
        - PLUS all other metadata that could be added by AO3 API
        - did not all text-data fics, only url-data fics--text data fics to be added in next notebook

<a id="imports"></a>
## 1. Imports

In [1]:
%run helpers.ipynb
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
cur_date = datetime.now().strftime('%m-%d-%y')

Success!


<a id="load_urls"></a>

## 2. Load in ALL urls (up until 01/13/23)

In [ ]:
# load in all urls from v7
with open("urlsOutput/v7_json/v7_authors.json", "r") as infile:
    v7_authors = json.load(infile) 
    
with open("urlsOutput/v7_json/v7_lookInto.json", "r") as infile:
    v7_lookInto = json.load(infile)  
    
with open("urlsOutput/v7_json/v7_others.json", "r") as infile:
    v7_others = json.load(infile) 
    
with open("urlsOutput/v7_json/v7_readUrls.json", "r") as infile:
    v7_read = json.load(infile) 
    
with open("urlsOutput/v7_json/v7_toReadUrls.json", "r") as infile:
    v7_toRead = json.load(infile) 
    
with open("urlsOutput/v7_json/v7_unsortedUrls.json", "r") as infile:
    v7_unsorted = json.load(infile) 

In [ ]:
with open("urlsOutput/chrome_1.json", "r") as infile:
    chrome = json.load(infile)

In [ ]:
# load in all urls collected BEFORE 1/13/23
# with open("urlsOutput/oldUrls_01-13-23.json", "r") as infile:
#     pre = json.load(infile) 
#     pre = pd.DataFrame(pre).drop_duplicates(subset=[0])[0].to_list() # de-dup pre list

with open("urlsOutput/pre.json", "r") as infile:
    pre = json.load(infile)

In [ ]:
# load in all urls collected on 1/13/23
with open("urlsOutput/readinglist_01-13-23.json", "r") as infile:
    cur = json.load(infile) 

<a id="4_dtb"></a>
## 3. Create 4 databases

In [ ]:
def toDatetime(x):
    if x is None: return x
    else: return datetime.strptime(x, '%m-%d-%y %H:%M:%S')

In [ ]:
dfCur = pd.DataFrame([])

In [ ]:
# define cur df - dates added, filter SECOND
from datetime import datetime

# load in cur
dfCur = pd.DataFrame(cur)

# convert dates
dfCur["date_added"] = dfCur["dateAdded"].apply(toDatetime)
dfCur["date_last_viewed"] = dfCur["dateLastViewed"].apply(toDatetime)
dfCur["smk_source"] = "safari"
dfCur["dtb_type"] = np.nan

# sort by date added
dfCur = dfCur.sort_values(by=["date_added"]).drop(columns=["dateAdded","dateLastViewed"])

# drop url duplicates (keep url added first)
dfCur = dfCur.drop_duplicates(subset=["url"], keep='first')

dfCur.head()

In [ ]:
# define pre df
dfPre = pd.DataFrame(pre).rename(columns={0:"url"})
dfPre["date_added"] = np.datetime64("NaT")
dfPre["date_last_viewed"] = np.datetime64("NaT")
dfPre["dtb_type"] = np.nan
dfPre["smk_source"] = "v8_local_url_files"

dfPre = dfPre.drop_duplicates(subset=["url"])

dfPre.head()

In [ ]:
def basicDF(myList, dtb_type, dateAdded=np.datetime64("NaT"), dateLastViewed=np.datetime64("NaT"), source="v7_sheets"):
    """
    Takes a list -> DF, adds dtb_type, date_added & date_last_viewed, de-dups
    """
    dfTemp = pd.DataFrame(myList).rename(columns={0:"url"}) # read in list
    
    # add cols
    dfTemp["dtb_type"] = dtb_type
    dfTemp["date_added"] = dateAdded
    dfTemp["date_last_viewed"] = dateLastViewed
    dfTemp["smk_source"] = source
    
    # de-dup
    dfTemp = dfTemp.drop_duplicates(subset=["url"])

    return dfTemp

 - 01/01/2022 is default date for all v7 stuff
 
 - priortize v7 if date is NOT 01-01-22 00:00:01

In [ ]:
# defines v7 df - filter FIRST, date_added = ~v8 date
date1 = datetime.strptime("01-01-22 00:00:01", '%m-%d-%y %H:%M:%S')

df_authors = basicDF(v7_authors, "author", date1)
df_lookInto = basicDF(v7_lookInto, "look_into", date1)
df_read = basicDF(v7_read, "read", date1)
df_toRead = basicDF(v7_toRead, "to_read", date1)
df_unsorted = basicDF(v7_unsorted, np.nan, date1)

# combines all v7 dfs
v7_df = pd.concat([df_authors, df_read, df_toRead, df_lookInto, df_unsorted]) # in order of priority, first kept
v7_df = v7_df.drop_duplicates(subset=["url"])

v7_df.head()

In [ ]:
# define chrome df - no dates
dfChrome = pd.DataFrame(chrome).rename(columns={0:"url"})
dfChrome["date_added"] = np.datetime64("NaT")
dfChrome["date_last_viewed"] = np.datetime64("NaT")
dfChrome["dtb_type"] = np.nan
dfChrome["smk_source"] = "chrome"

dfChrome = dfChrome.drop_duplicates(subset=["url"])

dfChrome.head()

### DATABASE
1. url - url
1. date_added - closet guess to when it was added, either from safari reading list data or version guess
    - v9 - 10-24-2022 00:00:01
    - v8 - 06-04-2022 00:00:01
    - v7 - 05/01/2021 00:00:01
1. date_last_viewed - date
    - only from safari reading list data
1. dtb_type - dtb type
    - read, toRead, lookInto, other, authors, NaN (unsorted)
1. smk_source - where I found url/where v9.2 is pulling it from
    - v8_local_url_files
    - v7_sheets
    - safari
    - chrome
    - txt_fic (not added yet)


<a id="4_to_1_dtb"></a>
## 4. Combine all 4 -> 1 (not added v1-6 dates)
- order: dfPre, dfChrome, dfCur, v7_df
    - dfPre + dfChrome, update (only urls will be concat'd)
    - +dfCur (add urls, overwrite with dateAdded & dateLastViewed)
    - rename v7_df.date_added -> v7_date_added
    - rename total.date_added -> cur_date_added
    - +v7_df (add urls, overwrite dtb_type, leaves dateLastViewed alone)
    - compare v7_date_added vs cur_date_added -> keep earliest one
    


In [ ]:
#dfPre + dfChrome
total = new_combine(dfPre, dfChrome)
total.smk_source.value_counts()

In [ ]:
# +dfCur
total = new_combine(total, dfCur)
total.smk_source.value_counts()

In [ ]:
# rename v7 & total's date_added
total = total.rename(columns={"date_added": "cur_date_added"})
v7_df = v7_df.rename(columns={"date_added": "v7_date_added"})
total = new_combine(total, v7_df)
total.smk_source.value_counts()

<a id='m1'></a>
## 5. Combined!  Now add/fill metadata cols:
- fill date_added
- add version nums

In [ ]:
# create total.date_added from the earliest date from 'cur_date_added' vs 'v7_date_added'
total["date_added"] = np.datetime64("NaT")

for ind in total.index:
    v7 = total.at[ind, "v7_date_added"]
    cur = total.at[ind, "cur_date_added"]
    newDate = np.datetime64("NaT")
    if pd.isnull(v7) and pd.isnull(cur): newDate= np.datetime64("NaT")
    elif not pd.isnull(v7) and not pd.isnull(cur): newDate= min([v7,cur])
    elif not pd.isnull(v7): newDate= v7
    else: newDate= cur
    
    total.at[ind, "date_added"] = newDate

total_2 = total.drop(columns=["v7_date_added","cur_date_added"])
total_2.head()

In [ ]:
# set all version nums (based on smk_source) (not added v1-6 or text fics)
total_2["version"] = 7

for ind in total_2.index:
    source = total_2.at[ind, "smk_source"]
    verNum = 7
    
    if source == "v8_local_url_files": verNum = 8
    elif source == "safari" or source == "chrome": verNum = 9
    
    total_2.at[ind, "version"] = verNum
    
total_2.head()

In [ ]:
# add date_added (by version number) for all non-date entities
# v9 - 10-24-2022 00:00:01
# v8 - 06-04-2022 00:00:01
# v7 - 05/01/2021 00:00:01

v9_date = datetime.strptime("10-24-22 00:00:01", '%m-%d-%y %H:%M:%S')
v8_date = datetime.strptime("06-04-22 00:00:01", '%m-%d-%y %H:%M:%S')
v7_date = datetime.strptime("05-01-21 00:00:01", '%m-%d-%y %H:%M:%S')

for ind in total_2.index:
    added = total_2.at[ind, "date_added"]
    if pd.isnull(added):
        new_date = 0
        source = total_2.at[ind, "smk_source"]
        if source == "v7_sheets": new_date = v7_date
        elif source == "v8_local_url_files": new_date = v8_date
        elif source == "safari" or source == "chrome": new_date = v9_date
        total_2.at[ind, "date_added"] = new_date
    
total_2.head()

<a id="seperate_others"></a>
## 6. Seperate AO3 works vs Others (non-ao3 & ao3 external works)

In [ ]:
# get others DF (non-AO3 and ao3 external works)
external = total_2.query("url.str.contains('archiveofourown.org/external_works/')")
dfOther = total_2.query("~url.str.contains('archiveofourown.org/')")
dfOther = pd.concat([external, dfOther]).reset_index(drop=True)

dfOther.head(2)

#### Others CHECKPOINT! others-0-all.csv (all non-AO3 urls & AO3 external works from all urls post 1/13/23, inclusive)

In [ ]:
# dfOther.to_csv("data-checkpoints/others-0-all_01-03-23.csv")

In [ ]:
# get non-other (aka all ao3) dtb
total_3 = total_2.query("url.str.contains('archiveofourown.org/')") \
        .query("~url.str.contains('archiveofourown.org/external_works/')") \
        .reset_index(drop=True)

total_3.head()

<a id='seperate_ao3_dtbs'></a>
## 7. Label ao3 work_type & separate into DTBs

In [ ]:
# add new columns for id & url_type
total_3["id"] = np.nan
total_3["url_type"] = np.nan

# label all cols
for ind in total_3.index:
    url = total_3.at[ind, "url"]
    
    data = getTypeAndId(url)
    wType = data[0]
    wId = data[1]
    
    total_3.at[ind, "url_type"] = wType
    total_3.at[ind, "id"] = wId

total_3.head()

In [ ]:
# reorganize cols order
total_3["location_found"] = "AO3"
all_ao3_links = total_3[["dtb_type","location_found","smk_source","version",
                   "date_added","date_last_viewed","url_type","id","url"]]
# all_ao3_links.to_csv(f"data-checkpoints/all_ao3_links_until_01-13-23__{cur_date}.csv")

#### ** All AO3 links including 1-13-23, inclusive 

<a id='clean_7'></a>
## 8. Clean up & customize the new 7 databases

In [ ]:
all_ao3_links = pd.read_csv("data-checkpoints/all_ao3_links_until_01-13-23__02-26-23.csv", 
                            index_col=0,
                            parse_dates = ['date_added','date_last_viewed'])

### 8.1) Clean & save Collections

In [ ]:
# separate collections from all_ao3_links
colDTB = all_ao3_links.query("url_type == 'collections'") \
                .reset_index(drop=True).rename(columns={"id":"name"}) \
                .drop(columns=["url_type"])

# correct data input mistake
colDTB.at[0,"name"] = "Canon_Divergence"

# write first col checkpoint
colDTB.to_csv(f"data-checkpoints/col-0-all_{cur_date}.csv")

colDTB.head()

#### ** Collections CHECKPOINT! - "col-0-all_02-26-23.csv" (all collections until this point)

### 8.2) Clean & save Searches

In [ ]:
# cleaning searches
searchDTB = all_ao3_links.query("url_type == 'search'") \
                .reset_index(drop=True).rename(columns={"id":"search_str"}) \
                .drop(columns=["url_type"])

# write first col checkpoint
searchDTB.to_csv(f"data-checkpoints/search-0-all_{cur_date}.csv")

searchDTB.head()

In [ ]:
# cleaning series
seriesDTB = all_ao3_links.query("url_type == 'series'") \
                .reset_index(drop=True) \
                .drop(columns=["url_type"])

# write first col checkpoint
seriesDTB.to_csv(f"data-checkpoints/series-0-all_{cur_date}.csv")

seriesDTB.head()

In [ ]:
# cleaning tags
tagsDTB = all_ao3_links.query("url_type == 'tags'") \
                .reset_index(drop=True) \
                .rename(columns={"id":"tag_str"}) \
                .drop(columns=["url_type"])

tagsDTB["tag_type"] = np.nan
tagsDTB["tag_type"] = tagsDTB["tag_type"].astype(str)

# write first col checkpoint
tagsDTB.to_csv(f"data-checkpoints/tags-0-all_{cur_date}.csv")

tagsDTB.head()

In [ ]:
# cleaning users
usersDTB = all_ao3_links.query("url_type == 'users'") \
                .reset_index(drop=True) \
                .rename(columns={"id":"user_name"}) \
                .drop(columns=["url_type"])

# write first col checkpoint
usersDTB.to_csv(f"data-checkpoints/users-0-all_{cur_date}.csv")

usersDTB.head()

In [ ]:
# cleaning fics
ficDTB = all_ao3_links.query("url_type == 'chapters' or url_type == 'works' or \
                url_type.str.contains('collections:')") \
                .reset_index(drop=True) 

# write first checkpoint
ficDTB.to_csv(f"data-checkpoints/fic-0-all_{cur_date}.csv")

ficDTB.head()

#### CHECKPOINT! - 0 (for all: collections, comments, search, tags, users)
- stored in data-checkpoints

<a id='fill_series'></a>
## 9. Fill seriesDTB (already de-dupped)

In [147]:
series_columns = {'dtb_type', 'location_found', 'smk_source', 'version', 'date_added',
       'date_last_viewed', 'id', 'url', 'name', 'creators', 'fandoms',
       'series_obj', 'date_obj_updated', 'description', 'notes', 'words',
       'complete', 'is_subscribed', 'series_begun', 'series_updated',
       'nbookmarks', 'nworks', 'work_list', 'is_restricted', 'not_found'}

<a id='fill_series.1'></a>
### 9.1 Define Series-filling Functions

In [392]:
# AO3.utils.limit_requests()

In [100]:
def getSeriesFandoms(series_obj):
    """
    Takes an AO3.Series object.
    Returns a list of all fandoms from all works in given series.
    """
    fandoms = []
    work_list = get_series_work_list(series_obj)
    for work in work_list:
        for fandom in work.fandoms:
            if fandom not in fandoms:
                fandoms.append(fandom)
    
    return fandoms

In [101]:
def get_series_work_list(series_obj):
    try: return [work for work in series_obj.work_list]
    except UnboundLocalError: return []

In [237]:
def series_row_complete(row):
    new_row = row.copy().drop(columns=['dtb_type','date_last_viewed', 
                                       'description', 'notes'])
    return len(np.where(pd.isnull(new_row))[1]) == 0

In [279]:
def fill_series_ao3_info(series_id, session, report=False):
    """
    Takes a series id (int).
    Returns a 1-row pandas DF populated by ao3 data from the given series id.
    """
    # initialize temp holder & Series obj
    single_series = pd.DataFrame({'id': [series_id]})
    series = AO3.Series(series_id, session=session)

    # write report info
    name = series.name
    creators = json.dumps([user.username for user in series.creators])
    fandoms = json.dumps(getSeriesFandoms(series))

    single_series['name'] = name
    single_series['creators'] = creators
    single_series['fandoms'] = fandoms
    if report: print(f"- Wrote '{name}' by {creators}\nin {fandoms}")

    # write remaining info
    single_series['series_obj'] = series
    single_series['date_obj_updated'] = datetime.now()
    
    single_series['description'] = series.description
    single_series['notes'] = series.notes
    single_series['words'] = series.words
    single_series['complete'] = series.complete
    single_series['is_subscribed'] = series.is_subscribed
    
    single_series['series_begun'] = series.series_begun
    single_series['series_updated'] = series.series_updated
    single_series['nbookmarks'] = series.nbookmarks
    single_series['nworks'] = series.nworks
    single_series['work_list'] = json.dumps([work.id for work in get_series_work_list(series)])
    
    single_series['is_restricted'] = series._soup.find("img", {"title": "Restricted"}) is not None
    single_series['not_found'] = False
    
    return single_series

# fill_series_ao3_info(1575793, my_session())

In [354]:
def fill_series_dtb(initial_series_dtb, session, update=False, report=False):
    """
    Takes a seriesDTB (pandas DataFrame, post series-0), an AO3 session, a Boolean to put in 'update' mode 
        (aka update all rows regardless of it they're already complete) and a Boolean to print report.
    Modifies given seriesDTB by filling it up with ao3 information.
    Returns nothing.
    """
    # find total number of series to fill
    total = max(initial_series_dtb.index)
    
    # ensure initial_series_dtb has all necessary columns
    for col in series_columns:
        if col not in initial_series_dtb.columns:
            initial_series_dtb[col] = np.nan

    # fill all series/rows in initial_series_dtb
    for ind in initial_series_dtb.index: 
        try: 
            # when not using full-report, alert at every 100 series
            if not report:
                if ind%100 == 0: 
                    print(f'- {ind}! (printed every 100)')
            
            # if series/row not entirely filled in OR we're updating the dtb 
            if (not series_row_complete(initial_series_dtb.iloc[[ind]])) or update: 
                # get series id
                series_id = initial_series_dtb.at[ind, "id"]
                if report: print(f"{ind}: [{(ind/total)*100:.2f}%] Filling for [{series_id}]")
                
                # get ao3 info
                series_ao3_info = fill_series_ao3_info(series_id, session, report=report)
                
                # update initial_series_dtb with series_ao3_info (new info will overwrite old info)
                series_ao3_info.index = [ind]
                initial_series_dtb.update(series_ao3_info, join='left', overwrite=True)
            
            # if series/row is satifactory
            else: 
                if report: print(f"{ind}: .")
        
        # if something goes wrong 
        except Exception as e:
            initial_series_dtb.at[ind, "not_found"] = True
            print(f"-- ERROR, {ind}: {initial_series_dtb.at[ind, 'id']}")
        
        # update temp csv w/ new row/series
        initial_series_dtb.to_csv("temp_series.csv")

    # Write finished series DTB to csv
    initial_series_dtb.to_csv("temp_series_final.csv")

    print("\nDONE!")


<a id='fill_series.2'></a>
### 9.2 Fill SeriesDTB

In [379]:
# most recent series data
seriesDTB = pd.read_csv("data-checkpoints/series-0-all_02-26-23.csv", 
                        index_col=0,
                        parse_dates=['date_added', 'date_last_viewed'])

In [349]:
# inititalize session
ss1 = my_session()

In [391]:
# fill_series_dtb(seriesDTB, ss1, update=False, report=True)

#### Series CHECKPOINT! (series-1, all & clean) (saved csv files in data-checkpoints)
- most recent: 03-06-23

<a id='fill_fics'></a>
## 10. Clean & prepare ficDTB

<a id='fill_fics_1'></a>
### 10.1 Clean prev data in ficDTB

In [ ]:
# add empty info cols
if False: 
    ficDTB["title"] = np.nan
    ficDTB["authors"] = np.nan
    ficDTB["fandoms"] = np.nan
    
    ficDTB["fic_obj"] = np.nan
    ficDTB["date_obj_updated"] = np.nan
    ficDTB["date_obj_updated"] = pd.to_datetime(ficDTB["date_obj_updated"])
    
    ficDTB["url_pseuds"] = np.nan
    ficDTB["col_work"] = np.nan

ficDTB.head(2)

In [ ]:
# add col_work collections
def getCol(x):
    if not "collections:" in x: return np.nan
    else:
        col = x.replace("collections:","")
        return col

# ficDTB["col_work"] = ficDTB["url_type"].apply(getCol)

In [ ]:
# update url_types (aka all 'collections:' -> col_work)
def x(x):
    if not "collections:" in x: return x
    else: return "col_work"

# ficDTB["url_type"] = ficDTB["url_type"].apply(x)

#### Fic CHECKPOINT! (fic-1, all & clean)

<a id='fill_fics_2'></a>
### 10.2 De-dup ficDTB

#### ficDTB Columns
- dtb_type: (read, to_read/NaN)
    - read
    - to_read
    - NaN
- smk_source: (v7, v8, safari, chrome)
    - v7_sheets
    - v8_local_url_files
    - chrome
    - safari
- version: 7-9 (earliest)
- date_added: (earliest)
    - date
    - NaT
- date_last_viewed (latest)
    - date
    - NaT
- url_type (works) (add others to url_psueds)
    - works
    - col_work
    - chapters
- id: num
- url: url
- col_work: list of col names
- url_psueds: list of non-main urls
    


In [ ]:
# read in ficDTB from checkpoint
ficDups = pd.read_csv("data-checkpoints/fic-1-clean_01-17-23_11-42-48.csv", index_col=0, 
                      parse_dates=["date_added","date_last_viewed"])
ficDups["url_psueds"] = json.dumps([])
ficDups.head() # fic_obj col no longer holds functional Fic objs

In [ ]:
def selectInfo(dtbCol, orderList):
    infos = wSlice[dtbCol].to_list()
    temp_info = np.nan
    for info in orderList:
        if info in infos: 
            temp_info = info
            break
            
    return temp_info

# all_smk_sources = ["v7_sheets","v8_local_url_files","safari","chrome"]
# selectInfo("smk_source", all_smk_sources)

In [ ]:
def emptyFicDTB():
    # initialize res
    res = pd.DataFrame(columns=['dtb_type',
                                 'smk_source',
                                 'version',
                                 'date_added',
                                 'date_last_viewed',
                                 'url_type',
                                 'id',
                                 'url',
                                 'col_work',
                                 'url_psueds'])
    
    # make right dtypes
    res["version"] = res["version"].astype("int")
    res["id"] = res["id"].astype("int")
    
    res["date_added"] = pd.to_datetime(res["date_added"])
    res["date_last_viewed"] = pd.to_datetime(res["date_last_viewed"])
    
    return res

In [ ]:
def mergeFics(ficId, ficDups):
    """
    Takes AO3 fic id, finds all fics with the same id in ficDups, merges them into one row.
    May need to update row all-value lists
    Returns that single row.
    """
    # get rows with same fic id
    wSlice = ficDups[ficDups["id"] == ficId].copy()
    # print(wSlice)
    
    if len(temp) > 1:
        # initialize temp result DF
        res = emptyFicDTB()

        # write dtb_type
        res.at[0,"dtb_type"] = selectInfo("dtb_type", all_dtb_types)

        # write smk_source
        res.at[0,"smk_source"] = selectInfo("smk_source", all_smk_sources)

        # write version
        res.at[0,"version"] = min(wSlice["version"].to_list())

        # write date_added
        res.at[0,"date_added"] = min(wSlice["date_added"].to_list())

        # write date_last_viewed
        res.at[0,"date_last_viewed"] = max(wSlice["date_last_viewed"].to_list())

        # write id
        res.at[0,"id"] = ficId

        # write col_work
        res.at[0,"col_work"] = json.dumps([x for x in wSlice["col_work"].to_list() if not pd.isnull(x)])

        # write url
        url = wSlice.iloc[0]["url"]
        res.at[0,"url"] = url

        # write url_type
        wType = getTypeAndId(url)[0]
        if "collections:" in wType: wType = "col_work"
        res.at[0,"url_type"] = wType
        
        # write url_psueds
        psueds = wSlice.url.drop_duplicates().to_list()[1:]
        if psueds is None: psueds = []
        res.at[0,"url_psueds"] = json.dumps(psueds)
        
    else:
        res = wSlice.iloc[0]
    
    return res
        
# mergeFics(19413088, ficDups)

In [ ]:
def deDupFics(ficDTB):
    """
    Takes a ficDTB, de-dups all fics in a way that preserves desired fic info order. Prints a progress bar.
    Returns a ficDTB with no duplicate fics & data preserved.
    """
    # fic info order (for dtb_types & sml_sources)
    all_dtb_types = ["read","to_read"]
    all_smk_sources = ["v7_sheets","v8_local_url_files","safari","chrome"]
    
    # make temp holder & get all ids
    temp_ficDTB = emptyFicDTB()
    ids = ficDTB.id.drop_duplicates().to_list()
    total = len(ids)
    
    # print report
    print(f"TOTAL: {total} ids to de-dup! {(total//100+1)*'|'}")
    print(f"{len(str(total))*' '}              PROGESS: ",end='')
    
    # add newly merged fics
    for i, wId in enumerate(ids):
        temp_row = mergeFics(wId, ficDTB)
        temp_ficDTB = pd.concat([temp_ficDTB, temp_row])
        
        # progress bar
        if (i%100) == 0: print('|', end="")

    return temp_ficDTB

In [ ]:
# De-dup ficDTB
ficDups = pd.read_csv("data-checkpoints/fic-1-clean_01-17-23_11-42-48.csv", index_col=0, 
                      parse_dates=["date_added","date_last_viewed"])
ficDups["version"] = ficDups["version"].astype("int")
ficDups["id"] = ficDups["id"].astype("int")

ficNoDups = deDupFics(ficDups)

In [ ]:
# write checkpoint
ficNoDups["version"] = ficNoDups["version"].astype("int")
ficNoDups["id"] = ficNoDups["id"].astype("int")

# ficNoDups.to_csv("data-checkpoints/fic-2-all_01-18-23.csv")

#### Fic CHECKPOINT! (fic-2, all & fic de-dupped)

<a id='fill_fics_3'></a>
### 10.3 Prepare ficDTB to be filled by AO3 API

In [ ]:
# read in checkpoint
ficDTB = pd.read_csv("data-checkpoints/fic-2-all_01-18-23.csv", index_col=0,
                     parse_dates=["date_added","date_last_viewed"]) \
                    .reset_index(drop=True) \
                    .drop(columns=['cur_chapter','notes'])
ficDTB.head(2)

In [ ]:
# define all col_name groups
cols = ['location_found','is_missing',
        'dtb_type','smk_source','version','date_added','date_last_viewed','url_type','id','url',
        'recced_from_collections','url_psueds']

tags = ['title','authors','fandoms','rating','categories','warnings',
        'relationships','characters','tags','series','collections']

meta = ['words','nchapters','expected_chapters', 'complete',
        'date_published','date_updated','date_edited',
        'language','restricted','metadata',]

text = ['summary','start_notes','end_notes','chapters','text',]

stats = ['kudos','comments','bookmarks','hits',]


tags.extend(meta)
tags.extend(text)
tags.extend(stats)

In [ ]:
# add cols to be used
if True: 
    for col in tags:
        ficDTB[col] = np.nan
        
    man_tags = ['fic_obj','date_obj_updated','is_subscribed','cur_chapter','notes']
    for tag in man_tags:
        ficDTB[tag] = np.nan
        
    ficDTB["location_found"] = 'AO3'
    ficDTB["is_missing"] = np.nan
    
    ficDTB = ficDTB.rename(columns={"col_work":"recced_from_collections"})

    ficDTB["date_obj_updated"] = pd.to_datetime(ficDTB["date_obj_updated"])
    ficDTB["date_published"] = pd.to_datetime(ficDTB["date_published"])
    ficDTB["date_updated"] = pd.to_datetime(ficDTB["date_updated"])
    ficDTB["date_edited"] = pd.to_datetime(ficDTB["date_edited"])

# reorganize new & old cols into desired order
all_cols = cols + man_tags + tags
ficDTB = ficDTB[all_cols]

ficDTB.head(2)

#### Fic CHECKPOINT! (fic-3-all, all data from prev versions + empty cols ready to be filled by AO3 API)
- most recent02-03-23

# And the notebook is done here!
- Next notebook is focused on competely compiling ALL fics into ficDTB & filling it